# Неделя 2. Четверг

## Линейная регрессия

* Ваша задача сегодня для лучшего понимания алгоритмов машинного обучения, реализовать свой класс Линейной регрессии на Python и `numpy` в частности.  

* на _train_ выборке алгоритму необходиму оубчиться, на _test_ выборке проверить свой результат. Метрика для проверки результата для линейной регрессии - _MSE_, метрики реализовать внутри класса.

* В качестве функции потерь, необходимо выбрать _MSE_ - для линейной регрессии. 

* Также необходиму пользователю Вашей модели предоставить возможность указать регуляризирующие коэффициенты и вид регуляризации('Ridge', 'Lasso', 'ElasticNet').  

* При инициализации класса пользователь указывает вид регуляризации, и коэффициенты регуляризации. Если вид не указан регуляризация отсутствует  


* После вы можете сравнить свой результат со стандартной Линейной регрессией, реализованной в _sklearn_

In [10]:
#Подгрузите необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [8]:
train = pd.read_csv('/Users/irinaevseeva/DS_bootcamp/week_5/train_flats.csv')
test = pd.read_csv('/Users/irinaevseeva/DS_bootcamp/week_5/test_flats.csv')

* __m2__ - площадь объекта (фича)
* __dist__ - удаленность объекта от центра города(фича)
* __price__ - цена (таргет)

0. Отнормируйте свои данные, используя [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Для линейных алгоритмов это очень важно  

In [15]:
ss = StandardScaler()

train[['m2', 'dist']] = ss.fit_transform(train[['m2', 'dist']])
test[['m2', 'dist']] = ss.transform(test[['m2', 'dist']])
train

,m2,dist,price
0,-0.467750,0.210353,40000
1,0.891581,-0.654054,190000
2,-0.959423,-0.687457,55000
3,-0.438828,0.906515,45000
4,2.279834,-0.377059,205000
...,...,...,...
157,-1.132954,0.063940,27000
158,-0.583437,0.019588,37000
159,2.742585,-1.198929,190000
160,0.255299,1.199076,50000


MSELoss = $\dfrac{1}{N} \sum_{i=1}^{N}(y_{act} - y_{pred})^2 = \dfrac{1}{N} \sum_{i=1}^{N}(y_{act} - (\omega_0 + x_{i1} \cdot \omega_1 + x_{i2} \cdot \omega_2 + ... + x_{in} \cdot \omega_n))^2$ 

*  Возьмем функцию потерь на одном объекте

$L = (y_{act} - y_{pred})^2$  
  
$y_{act} - $ реальное значение, которое принимает наша величина

$y_{pred} - $ значение, которое будет предсказывать наша модель


Наше желание, чтобы модель, на каждом элементе выборки предсказывала значение как можно ближе к реальному

* Как это сделать?

Мы знаем, что в случае линейной регрессии наше предсказание строится как  

$y_{pred} = (\omega_0 + x_{1} \cdot \omega_1 + x_{2} \cdot \omega_2 + ... + x_{n} \cdot \omega_n)$  

$w_0, w_1, w_2, ..., w_n$ - Параметры, которые мы могли бы настроить!  

Итоговая функция потерь на одном объекте:

$L(\vec{w}) = (y_{act} - (\omega_0 + x_{i1} \cdot \omega_1 + x_{i2} \cdot \omega_2 + ... + x_{in} \cdot \omega_n))^2$

$L(\vec{w})$ - сложная функция, которая состоит из следующих функциональных преобразований:  
* возведения в квадрат
* домножения наших $\omega$ на константу - входные данные $x_1, x_2, ..., x_n$. Да да, именно они являются константами

(_Смотреть выше расписанную формулу_)



1. Посчитать для $L(\vec{w}) = (y_{act} - (\omega_0 + x_{i1} \cdot \omega_1 + x_{i2} \cdot \omega_2 + ... + x_{in} \cdot \omega_n))^2$ сложную частную производную по $w_1$.

Как будет отличаться частная производная для $w_2, w_3, ..., w_n$?

2. Посчитать частную производную для $w_0$. (Свободного члена)

Математически градиент готов, останется обернуть его в алгоритм градиентного спуска и на реальных данных, где у нас ни один объект, а много сразу.
Единственной разницей того, что объектов много сразу, мы будем минимизировать функцию потерь в среднем на всех элементах.

Формула для одного объекта была бы такой:
    
$\vec{w_{new}} = \vec{w_{old}} - lr * grad L(\vec{w_{old}})$  

Для всех объектов: 

1. Высчитывается градиент на каждом из ваших объектов(везде получаются разные $grad L(\vec{w_{old}})$ - так как у каждого объекта свои $y, x_1, x_2, ...$).
2. Берется средний $\vec{\omega_{old}}$ - по нему вычисляется новый $\vec{w_{new}}$ у модели

Итого:

$\vec{w_{new}} = \vec{w_{old}} - lr * mean(grad L(\vec{w_{old}}))$

3. Создать класс LinReg. При инициализации дать возможность указать learning_rate, кол-во входных фичей(n). Записать эту информацию в атрибуты класса

In [18]:
class LinReg:
    def __init__(self, learning_rate=0.1, n_inputs=1):
         self.learning_rate = learning_rate
         self.n_inputs = n_inputs

4. Создать случайную инициализцию необходимых $\omega$ (Их будет n+1). Инициализируйте их равномерным распределением w1, w2, ..., wn = положите в атрибут - coef_, w0(свободный член) положите в атрибут intercept_

In [22]:
class LinReg:
     def __init__(self, learning_rate, n_inputs):
         self.learning_rate = learning_rate
         self.n_inputs = n_inputs
         self.coef_ = np.random.normal(-5, 5, size=n_inputs)
         self.intercept_ = np.random.normal(-5, 5, size = 1)

5. Опишите метод fit, который будет принимать на вход X, y (X - данные x1, x2, ..., xn, y - это $y_{act})$ и высчитывать с помощью градиентного спуска самые оптимальные параметры w0, w1, w2, ..., wn. Которые будут хранится в атрибутах coef_ и intercept_

In [29]:
class LinReg:
    def __init__(self, learning_rate=0.01, n_inputs=2, epoch = 10):
        self.learning_rate = learning_rate
        self.n_inputs = n_inputs
        self.coef = np.random.normal(-5, 5, size = n, epoch = 10)
        self.intersept_ = None

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        n_samples, n_features = X.shape

        # init parameters
        self.coef = np.zeros(n_features)
        self.intersept_ = 0

        # gradient descent
        for _ in range(self.n_iters):
            # y_predicted = np.dot(X, self.coef) + self.intersept_
            y_predicted = self.predict(X)
            # compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # update parameters
            self.coef -= self.lr * dw
            self.intersept_ -= self.lr * db

    def predict(self, X):
        y_approximated = np.dot(X, self.coef) + self.intersept_
        return y_approximated


6. Опишите метод predict, который будет предсказывать для новых точек в дальнейшем их y_pred

In [7]:
X = sc_test[['m2', 'dist']]

res = lin.predict(X)
# pd.DataFrame(res)

7. Сравните результат с линейной регрессией в sklearn. 

In [8]:
from sklearn.linear_model import LinearRegression

X = sc_train[['m2', 'dist']]
y = sc_train['price']

lr = LinearRegression()
lr.fit(X, y)
lr.coef_, lr.intercept_, lin.coef_, lin.intercept_

8. Напшите метод _score_. Который принимает данные _X_ и _y_  и высчитывает функционал качества, можно оставить тот же [MSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error). Он нам понадобится для оценки качества работы алгоритмы на тестовых данных

In [9]:
from sklearn.metrics import mean_squared_error

X_test = sc_test[['m2', 'dist']]
y_test = sc_test['price']

our_res = lin.score(X_test, y_test)
sk_res = mean_squared_error(y_test, lr.predict(X_test))

print(f"Our result: {round(our_res, 5) : >20} \nSklearn's result: {round(sk_res, 5) : >14}")

9. Посчитайте ваш _score_ и линейной регрессии из sklearn для тестового набора данных

10. Нарисуйте 3D график, на котором будет следующее:

* ось X и Y - [['m2', 'dist']]
* ось Z -  price
* через scatter все элементы выборки. Красными точками train, Синими test
* Линейную плоскость предсказания

10*. Добавьте возможность пользователю добавлять реугляризацию модели. Это не привносит больших изменений. Немного пмоеняется функция потерь и как следствие градиент

In [ ]:
# class LinReg:
#     def __init__(self, learning_rate, n_inputs, reg_type='Ridge', alpha=0.2):
#         self.learning_rate = ...
#         self.n_inputs = ...
#         self.coef_ = ...
#         self.intercept_ = ...
        
#     def fit(self, X, y):
#         pass

#     def predict(self, X):
#         pass